In [2]:
import pandas as pd
import numpy as np
import sys
import os

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.default_inference import DefaultInference
import plotly #Plotting

#GLM == General Linear Models
import scipy
import statsmodels

#Find the utils Directory
sys.path.append(os.path.abspath("../../"))
from src.utils.ConvertTextToCsv import TextToCsv

In [3]:
df_txt = pd.read_csv("data_mrna_seq_v2_rsem.txt", header=None)
print(df_txt)

                                                       0
0      Hugo_Symbol\tEntrez_Gene_Id\tTCGA-A1-A0SB-01\t...
1      UBE2Q2P2\t100134869\t14.3935\t11.3241\t4.4426\...
2      HMGB1P1\t10357\t116.3870\t60.2630\t153.1452\t1...
3      LOC155060\t155060\t279.7612\t83.6986\t74.7018\...
4      RNU12-2P\t26823\t0.4505\t0.3308\t0.0000\t0.000...
...                                                  ...
20436  ZYX\t7791\t6186.7327\t3559.6725\t3007.8157\t53...
20437  ZZEF1\t23140\t1931.2986\t1278.9678\t926.3677\t...
20438  ZZZ3\t26009\t1436.1978\t1195.6000\t1075.4422\t...
20439  TPTEP1\t387590\t552.3144\t86.0144\t866.1456\t5...
20440  AKR1C6P\t389932\t0.0000\t0.0000\t0.0000\t0.856...

[20441 rows x 1 columns]


In [19]:
df_csv = TextToCsv("data_mrna_seq_v2_rsem.txt")
print(df_csv.head())

Shape of the CSV: (20440, 819)
  Hugo_Symbol  Entrez_Gene_Id  TCGA-A1-A0SB-01  TCGA-A1-A0SD-01  \
0    UBE2Q2P2       100134869          14.3935          11.3241   
1     HMGB1P1           10357         116.3870          60.2630   
2   LOC155060          155060         279.7612          83.6986   
3    RNU12-2P           26823           0.4505           0.3308   
4        SSX9          280660           0.0000           0.0000   

   TCGA-A1-A0SE-01  TCGA-A1-A0SF-01  TCGA-A1-A0SH-01  TCGA-A1-A0SI-01  \
0           4.4426          10.7401           3.0048           2.9782   
1         153.1452         141.1933          79.8003          63.5491   
2          74.7018         314.4482          95.7054         149.7940   
3           0.0000           0.0000           0.0000           0.2943   
4           0.0000           0.0000           0.0000           0.0000   

   TCGA-A1-A0SJ-01  TCGA-A1-A0SK-01  ...  TCGA-LL-A5YM-01  TCGA-LL-A5YN-01  \
0           4.9419          28.8560  ...         

Eliminate The values that are 0 in the rows

In [21]:
df_counts = df_csv.drop(columns=["Hugo_Symbol", "Entrez_Gene_Id"])

df_counts = df_counts.T

df_counts = df_counts.astype(int)
#Filtrate
df_counts = df_counts.loc[:, df_counts.sum(axis=0) >= 10]
print(df_counts.head())

                 0      1      2      3      4      5      6      7      \
TCGA-A1-A0SB-01     14    116    279      0      0      0      0      1   
TCGA-A1-A0SD-01     11     60     83      0      0      0      0      4   
TCGA-A1-A0SE-01      4    153     74      0      0      0      0      5   
TCGA-A1-A0SF-01     10    141    314      0      0      0      2      9   
TCGA-A1-A0SH-01      3     79     95      0      0      0      0      3   

                 8      9      ...  20430  20431  20432  20433  20434  20435  \
TCGA-A1-A0SB-01      0      0  ...     95    519   1415     19   1364   6186   
TCGA-A1-A0SD-01      0      0  ...     96    578   1225     33    868   3559   
TCGA-A1-A0SE-01      0      0  ...     95    726   1018     57    960   3007   
TCGA-A1-A0SF-01      0      0  ...     74    533   1053     94    881   5343   
TCGA-A1-A0SH-01      0      0  ...     87    581    801     19   1353   5464   

                 20436  20437  20438  20439  
TCGA-A1-A0SB-01   1931

In [15]:
res = df_csv.mask(df_csv.values == 0)
df_csv = res.dropna()
print(df_csv)


      Hugo_Symbol  Entrez_Gene_Id  TCGA-A1-A0SB-01  TCGA-A1-A0SD-01  \
1         HMGB1P1           10357         116.3870          60.2630   
2       LOC155060          155060         279.7612          83.6986   
14        HSPB1P1          653553          43.6986         226.9457   
17         GTPBP6            8225         392.3865         271.2762   
19           A1BG               1          49.1992         142.2976   
...           ...             ...              ...              ...   
20434      ZYG11B           79699        1364.5681         868.0837   
20435         ZYX            7791        6186.7327        3559.6725   
20436       ZZEF1           23140        1931.2986        1278.9678   
20437        ZZZ3           26009        1436.1978        1195.6000   
20438      TPTEP1          387590         552.3144          86.0144   

       TCGA-A1-A0SE-01  TCGA-A1-A0SF-01  TCGA-A1-A0SH-01  TCGA-A1-A0SI-01  \
1             153.1452         141.1933          79.8003          63.5

In [ ]:

#N of Cpu
inference = DefaultInference(n_cpus=2)
metadata = pd.DataFrame(index=df_counts.index)
print(metadata)

In [ ]:
deseqDataset = DeseqDataSet(
    counts=df_counts,
    metadata=metadata,
    design="~condition",
    refit_cooks=True,
    inference=inference
)

deseqDataset.deseq2()